### Create an assistant

In [4]:
from openai import OpenAI

key = your keyclient = OpenAI(api_key=key)
assistant = client.beta.assistants.create(
    name = "Expert in Requirements Engineering",
    instructions="You are an expert in Requirements Engineering. You are provided with abstracts of papers from IEEE International Requirements Engineering Conference. You would like to desing an ontology to describe the concepts and the relationships among concepts in your domain. Competency questions specify what knowledge has to be entailed in the ontology and thus can be seen as a set of requirements on the content as well as a way of scoping and delimiting the subject domain that has to be represented in the ontology.",
    tools=[{"type": "file_search"}],
    model="gpt-4-turbo"
)

### Upload files and add them to a Vector Store

In [5]:
import os

def get_file_paths(folder_path):
    file_paths = []
    # Iterate through all files in the folder
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            # Get the absolute path of the file
            file_path = os.path.join(root, file_name)
            # Append the file path to the list
            file_paths.append(file_path)
    return file_paths

# create a Vector Store named vu-ut
vector_store = client.beta.vector_stores.create(name="vu-ut")

folder = "abstracts"
file_paths = get_file_paths(folder)
file_streams = [open(path, 'rb') for path in file_paths]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_streams
)

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=45, failed=0, in_progress=0, total=45)


### Update the assistant to use the new Vector Store

In [9]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

### Create a thread

In [10]:
thread = client.beta.threads.create()

### Add a message to the thread

In [11]:
def get_response(query,client,assistant,thread):
    # add message to the thread
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=query
    )

    # create a run
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    # get messages
    messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
    message_content = messages[0].content[0].text
    annotations = message_content.annotations
    citations = []
    for index, annotation in enumerate(annotations):
        message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
        if file_citation := getattr(annotation, "file_citation", None):
            cited_file = client.files.retrieve(file_citation.file_id)
            citations.append(f"[{index}] {cited_file.filename}")

    # print(message_content.value)
    # print("\n".join(citations))
    return message_content.value,citations

#### query 1

In [14]:
query = "What is Requirements Engineering?"
response = get_response(query,client,assistant,thread)
print(f"query: {query}")
print(f'response: {response[0]}')
# print(f"reference: {response[1]}")

query: What is Requirements Engineering?
response: Requirements Engineering (RE) is the process of defining, documenting, and maintaining requirements in the engineering design process. It involves various stages such as eliciting requirements from stakeholders, analyzing these requirements to determine feasibility and implications, specifying and documenting the requirements in detail, validating and verifying the adequacy of the documented requirements, and managing the requirements as the system is developed and after it is deployed. This process is crucial in ensuring that the developed system or product meets the intended purpose and needs of its intended users and stakeholders.


#### query 2

In [15]:
query = "Now you are developing an ontology in Requirements Engineering. Based on the abstracts of papers from the IEEE International Requirements Engineering Conference, derive 5 competency questions."
response = get_response(query,client,assistant,thread)
print(f"query: {query}")
print(f'response: {response[0]}')
# print(f"reference: {response[1]}")

query: Now you are developing an ontology in Requirements Engineering. Based on the abstracts of papers from the IEEE International Requirements Engineering Conference, derive 5 competency questions.
response: Based on the abstracts from the IEEE International Requirements Engineering Conference, here are five competency questions that an ontology in Requirements Engineering might address:

1. **How can Trace Link Recovery approaches use semantic relation graphs to enhance the automation and precision in linking requirements**?
   - This question is driven by the need to improve Trace Link Recovery methods using semantic representations that automate the creation of trace links between requirements[0].

2. **What methods are effective in automatically classifying and tracing non-functional requirements (NFRs) in software systems**?
   - Since NFRs are critical for software stability and often overlooked during early development phases, understanding methods to automatically extract and